<a href="https://colab.research.google.com/github/hanilRyoo/gpt-based-my-own-chatbot/blob/main/pdf_dataset_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai langchain langchainhub pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.


In [ ]:
import os
from google.colab import drive

# This line mounts your Google Drive to the Colab notebook.
drive.mount('/content/gdrive', force_remount=True)

# Define the root directory and the specific folder path.
root_dir = "/content/gdrive/My Drive/"
pdf_folder_path = os.path.join(root_dir, '교양과목/data/')

# List the contents of the specified folder.
files = os.listdir(pdf_folder_path)
print(files)


Mounted at /content/gdrive
['2023학년도 대학생활안내.pdf', '생활의화학.pdf', '환경과학.pdf', '지구의이해.pdf', '과학사의이해.pdf', '미디어문화와디자인.pdf', '포스트휴먼과상상력.pdf', '36.5도의물리학.pdf', '세상을바꾸는커뮤니케이션.pdf', '빅데이터의과학적탐구.pdf', '삶속의생명과학.pdf', '대학과학문.pdf', '철학과삶.pdf', '서양문학기행.pdf', '프랑스감성과문학.pdf', '예술감성과미학.pdf', '한국인의삶과문학.pdf', '미디어로보는일본문학과감성.pdf', '문화야놀자_기억과놀이.pdf', '이큐독문학.pdf', '사랑과폭력의인문학.pdf', '디지털시대로컬문화큐레이팅.pdf', '중국문학의숲에서거닐다.pdf', '한국사.pdf', '현대사회의이해.pdf', '분쟁의세계지리.pdf', '공동체와현대행정.pdf', '동양의역사와문화.pdf', '서양의역사와문화.pdf', '공존과협력의심리.pdf', '인류진화와고대문명.pdf', '민주주의와공동체.pdf', '결혼과가족의이해.pdf', '공동체와담론.pdf', '윤리적사유와공동체.pdf', '세계속의프랑

In [ ]:
len(files)

200

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = '자신의 gpt api'

from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()
#docs[0].page_content[:100]

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

In [ ]:
!pip install -q langchain openai tiktoken chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.10 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 임베딩
embeddings = OpenAIEmbeddings()
# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings)
# retriever 가져옴
retriever = docsearch.as_retriever(search_kwargs={"k": 4})

In [ ]:
# docs = retriever.get_relevant_documents("소프트웨어공학과 교양 과목")
# print('유사 문서 개수 :', len(docs))
# print('--' * 20)
# print('첫번째 유사 문서 :', docs[0])
# print('--' * 20)
# print('각 유사 문서의 문서 출처 :')
# for doc in docs:
#     print(doc.metadata["source"])

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4-1106-preview",  max_tokens=4000, temperature=0.0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False)

#gpt-3.5-turbo
#gpt-4-1106-preview

In [ ]:
# input_text = "소프트웨어공학과 교양필수 과목은 뭐가있어?"
# chatbot_response = qa_chain(input_text)
# chatbot_response

In [ ]:
!pip install kaleido cohere

In [ ]:
#!pip uninstall -q gradio --use-deprecated=legacy-resolver

In [ ]:
!pip install -q gradio==3.45.0

In [ ]:
#!pip install --upgrade gradio

In [ ]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks(theme='JohnSmith9982/small_and_pretty') as demo:

    with gr.Row():
        gr.HTML("""
            <div style="float:left">
                <img alt width="130" height="130" margin= "0" src="https://drive.google.com/uc?id=1oy23oCCrUENCnNThgByq3ZYNkGNgMnI3" align=middle >
            </div>
            <div>
                <h1>&ensp;</h1>
                <h1>안녕하세요?</h1>
                <h1>저는 AI 챗봇 용봉이에요</h1>
            </div>
        """)
    chatbot = gr.Chatbot(label="대학생활 도우미 용봉이")
    msg = gr.Textbox(label="질문해주세요!")
    examples = gr.Examples(examples=["소프트웨어공학과 교양필수 과목은 뭐야?", "소프트웨어공학과 졸업요건은 뭐야?","일반화학은 뭘 배워?","휴학 신청하려면 어떻게 해?"],
                           inputs=[msg])

    clear = gr.Button("대화 초기화")

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>